In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12054101256808518644, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10850542912537822897
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8440057814837351615
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14640891840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17228556216949611299
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [46]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Conv2D,Dense,Flatten,Dropout,BatchNormalization,MaxPooling2D
from keras.models import Model,load_model
from sklearn.model_selection import train_test_split
import os
from sklearn.model_selection import StratifiedKFold
from skimage import color
import pickle
from multiprocessing import Queue
%load_ext google.colab.data_table
import datetime
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=123)

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [3]:
!pip show tensorflow

Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wheel, google-pasta, numpy, gast, h5py, tensorboard, keras-preprocessing, termcolor, protobuf, opt-einsum, wrapt, six, scipy, grpcio, absl-py, astunparse, tensorflow-estimator
Required-by: fancyimpute


In [3]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

%cd 'drive/My Drive'


Mounted at /content/drive/
/content/drive/My Drive


In [4]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [ ]:
for a in x_train:
  print(max(a[56]),min(a[56]))

In [5]:
num_neg = len(os.listdir('/content/drive/My Drive/X-ray/chest_xray/chest_xray/train/NORMAL'))
num_pos = len(os.listdir('/content/drive/My Drive/X-ray/chest_xray/chest_xray/train/PNEUMONIA'))
total_num = num_neg+num_pos

In [6]:
train_datagen = ImageDataGenerator(rotation_range=360,
                             width_shift_range = 0.5,
                             height_shift_range = 1.0,
                             horizontal_flip = True,
                             vertical_flip = True,
                             rescale = 1./255.)
val_datagen = ImageDataGenerator(rescale=1./255.)
no_batch = 64

In [7]:
inputs = tf.keras.Input(shape=(128,128,1))
#feature extract 1
conv1 = Conv2D(16,kernel_size=3,activation='relu')(inputs)
conv2 = Conv2D(16,kernel_size=5,activation='relu')(conv1)
norm = BatchNormalization()(conv2)
dropout = Dropout(0.25)
conv3 = Conv2D(32,kernel_size=3,strides=2,activation='tanh')(norm)
conv4 = Conv2D(64,kernel_size=3,activation='relu')(conv3)
norm1 = BatchNormalization()(conv4)
flatten = Flatten()(norm1)
dense1 = Dense(64,activation='relu')(flatten)
dense2 = Dense(128,activation='relu')(dense1)
output = Dense(1,activation = 'sigmoid')(dense2)

model = Model (inputs = inputs,outputs=output)

In [8]:

weight_for_0 = (1 / num_neg)*(total_num)/2.0 
weight_for_1 = (1 / num_pos)*(total_num)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

In [9]:
model.compile(optimizer = 'adam',metrics = ['binary_accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],loss = 'binary_crossentropy')
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 126, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 16)      6416      
_________________________________________________________________
batch_normalization (BatchNo (None, 122, 122, 16)      64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 58, 58, 64)       

In [10]:
recall = 'recall'
total_list = list()
fold = 0
with tf.device('/device:GPU:0'):
  for train,val in skf.split(x_train,y_train):
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor = recall,patience=5,verbose=1,restore_best_weights=True)
    reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor=recall,patience=2,verbose=1,cooldown=1)
    print(fold)
    model.fit(train_datagen.flow(x_train[train],y_train[train],
              batch_size = no_batch),
              epochs=25,
              verbose=1,
              callbacks=[earlystopping,reduceLR],
              class_weight = class_weight)
    scores = model.evaluate(val_datagen.flow(x_train[val],y_train[val]),return_dict=True)
    fold += 1
    print(scores)
    total_list.append(scores)
#final_eval
print('--------------------------------------------/n')
print('done training')


0
Epoch 1/25
74/74 [==============================] - 8s 103ms/step - loss: 2.7644 - binary_accuracy: 0.6842 - precision: 0.8557 - recall: 0.6909
Epoch 2/25
74/74 [==============================] - 7s 101ms/step - loss: 2.0256 - binary_accuracy: 0.7186 - precision: 0.8764 - recall: 0.7227
Epoch 3/25
74/74 [==============================] - ETA: 0s - loss: 1.5637 - binary_accuracy: 0.7062 - precision: 0.8804 - recall: 0.6992
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
74/74 [==============================] - 7s 101ms/step - loss: 1.5637 - binary_accuracy: 0.7062 - precision: 0.8804 - recall: 0.6992
Epoch 4/25
74/74 [==============================] - 7s 101ms/step - loss: 0.9304 - binary_accuracy: 0.7424 - precision: 0.8995 - recall: 0.7350
Epoch 5/25
74/74 [==============================] - ETA: 0s - loss: 0.6719 - binary_accuracy: 0.7710 - precision: 0.9174 - recall: 0.7599
Epoch 00005: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e

In [15]:
xray_model = model

In [9]:
xray_model = load_model('xray_model.h5')

In [16]:
x_ray_url= 'https://www.radiologymasterclass.co.uk/images/quizzes/Quiz-Images-Chest-1/question_1.jpg'
path = tf.keras.utils.get_file('xray',x_ray_url)

img = keras.preprocessing.image.load_img(path,color_mode='grayscale',target_size=(128,128))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = img_array/255.
expanded = tf.expand_dims(img_array,0)

prediction = xray_model.predict(expanded)

In [17]:
score=tf.math.sigmoid(prediction)

In [18]:
classes = ['NORMAL', 'PNEUMONIA']

In [19]:
print(f"this image most likely belongs to {classes[np.argmax(score)]} lung with a {np.max(score)*100} percent confidence")

this image most likely belongs to NORMAL lung with a 50.000590085983276 percent confidence


In [38]:
model.save('xray_model.h5')

In [20]:
!pip install lime

     |████████████████████████████████| 276kB 9.2MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283845 sha256=e627441eaa9e262a6e74835f6011f3c13b59846b46bbf2de10595874135da1f2
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [21]:
from lime import lime_image

In [22]:
%load_ext autoreload
%autoreload 2

In [23]:
explainer = lime_image.LimeImageExplainer()


In [24]:
rgb_img = tf.image.grayscale_to_rgb(expanded)

In [25]:
rgb_img.shape

TensorShape([1, 128, 128, 3])

In [26]:
array_rgb =rgb_img.numpy()

In [27]:
transformed_img=np.reshape(array_rgb,(128,128,3))

In [28]:
transformed_img.shape

(128, 128, 3)

In [35]:

def grey_model (rgb_img,model):
  grey_img = color.rgb2gray(rgb_img)
  return model.predict(grey_img)

In [40]:
explanation=explainer.explain_instance(transformed_img,grey_model(transformed_img,xray_model),labels=2,hide_color=0,top_labels=2,num_samples=16384)

ValueError: ignored

In [52]:
xray_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 126, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 16)      6416      
_________________________________________________________________
batch_normalization (BatchNo (None, 122, 122, 16)      64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 58, 58, 64)       

In [54]:
tf.keras.models.save_model(xray_model,filepath = 'xray_model',save_format='tf')

FailedPreconditionError: ignored